# Preview Quiz prompt

In [1]:
from pydantic import BaseModel
from openai import OpenAI
from openai.lib._parsing import type_to_response_format_param
import json
from getpass import getpass

openai_api_key = getpass("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [2]:
from jinja2 import Template

prompt_template = Template("""
[PURPOSE]
Preview lesson content through quizzes to give a sneak peek before the actual lesson.
The quizzes must ask key expressions/the most structurally important words themselves written in or based on the GOAL/DESCRIPTION.

[QUIZ GUIDELINES]
- Create two preview quizzes per topic. One from KEY_EXPRESSION, the other from SUB_EXPRESSIONS.
- Put blanks on the KEY_EXPRESSION and SUB_EXPRESSION itself, no elsewhere.
- No blanks or tildes in Korean translation
- No tildes in the quiz.
- Make sure you have NO multiple "equally valid" answers
- Blanks must directly ask the key expressions/the most meaningful words themselves based on the DESCRIPTION.
- Answer index = the position number of the answer.
- Ensure that the quiz is a verb, adjective, or a key phrase from the lesson.
- No proper nouns can be tested.
- Create SAMPLE SCRIPT for each topic at the end of each topic.


[POLICY]
Each quiz needs:
- A blank (test key patterns itself)
- Complete Korean translation (no blanks or tildes)
- Three options (one correct, two wrong)
- No multiple "equally valid" answers/options
- Wrong options must be distinctly wrong.
- Shuffle the order of answer choices='answer indexes (1-3)' to avoid predictability.
- SAMPLE SCRIPT must not be the same as the quizzes. Change one or two words to create a dialogue of different topic, but maintain the structure.

[Example]
```
Input: 
    {
    "DESCRIPTION": "This is my friend, ~와 같은 표현을 사용해 사람을 소개하기.",
    "KEY_EXPRESSION": "A: This is my friend, Jake.",
    "SUB_EXPRESSION": [
      "A: Let me introduce my friend, ~. A: Meet my friend, ~."
    ]
  }

Output:
    {
    "goal": [
      "다른 사람을 소개하는 표현을 사용할 수 있어요.",
      "This is my friend, ~과 같은 표현을 사용하여 친구를 소개할 수 있어요."
    ],
    "preview_quizzes": [
      {
        "fill_in_blank_quiz": "___ is my friend, Jennie.",
        "korean_translation": "이쪽은 내 친구 Jennie야.",
        "options": ["I", "They", "This"],
        "answer": "This",
        "answer_index": 3
      },
      {
        "fill_in_blank_quiz": "Let me ___ my friend, Sarah.",
        "korean_translation": "내 친구 Sarah를 소개할게.",
        "options": ["introduce", "inter", "intro"],
        "answer": "introduce",
        "answer_index": 2
      }
    ]
    "sample_script" : [
      {
        "sample_script_eng" : "A: This is my friend, Emma. B: Nice to meet you, Emma! A: Let me introduce my friend, David. B: Hey, David! Glad to meet you.",
        "sample_script_kor" : "A: 얘는 내 친구, 엠마야! B: 반가워, 엠마! A: 내 친구 데이빗을 소개할게. B: 안녕, 데이빗! 만나서 반가워."
      }                       
    ]
  }
}
```

[Input]
```
DESCRIPTION: {{description}}
KEY EXPRESSION: {{key_expression}}
SUB EXPRESSION: {{sub_expression}}
```

""")

In [74]:
prompt = prompt_template.render(
    description = "동격의 of, that",
    key_expression = "The fact of his success surprised everyone.",
    sub_expression = [
        "The fact that he won surprised everyone. We discussed the possibility of traveling abroad."
    ]
)

In [13]:
print(prompt)


[PURPOSE]
Preview lesson content through quizzes to give a sneak peek before the actual lesson.
The quizzes must ask key expressions/the most structurally important words themselves written in or based on the GOAL/DESCRIPTION.

[QUIZ GUIDELINES]
- Create two preview quizzes per topic. One from KEY_EXPRESSION, the other from SUB_EXPRESSIONS.
- Put blanks on the KEY_EXPRESSION and SUB_EXPRESSION itself, no elsewhere.
- No blanks or tildes in Korean translation
- No tildes in the quiz.
- Make sure you have NO multiple "equally valid" answers
- Blanks must directly ask the key expressions/the most meaningful words themselves based on the DESCRIPTION.
- Answer index = the position number of the answer.
- Ensure that the quiz is a verb, adjective, or a key phrase from the lesson.
- No proper nouns can be tested.
- Create SAMPLE SCRIPT for each topic at the end of each topic.


[POLICY]
Each quiz needs:
- A blank (test key patterns itself)
- Complete Korean translation (no blanks or tildes)


In [5]:
class PreviewQuiz(BaseModel):
    fill_in_blank_quiz : str
    korean_translation : str
    options : list[str]
    answer : str
    answer_index : int

class SampleScript(BaseModel):
    sample_script_eng : str
    sample_script_kor : str

class QuizResponse(BaseModel):
    goal : list[str]
    preview_quizzes : list[PreviewQuiz]
    sample_script : list[SampleScript]

In [6]:
response_format = type_to_response_format_param(QuizResponse)

In [7]:
response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'$defs': {'PreviewQuiz': {'properties': {'fill_in_blank_quiz': {'title': 'Fill In Blank Quiz',
       'type': 'string'},
      'korean_translation': {'title': 'Korean Translation', 'type': 'string'},
      'options': {'items': {'type': 'string'},
       'title': 'Options',
       'type': 'array'},
      'answer': {'title': 'Answer', 'type': 'string'},
      'answer_index': {'title': 'Answer Index', 'type': 'integer'}},
     'required': ['fill_in_blank_quiz',
      'korean_translation',
      'options',
      'answer',
      'answer_index'],
     'title': 'PreviewQuiz',
     'type': 'object',
     'additionalProperties': False},
    'SampleScript': {'properties': {'sample_script_eng': {'title': 'Sample Script Eng',
       'type': 'string'},
      'sample_script_kor': {'title': 'Sample Script Kor', 'type': 'string'}},
     'required': ['sample_script_eng', 'sample_script_kor'],
     'title': 'SampleScript',
     'type': 'object',
     '

In [8]:
def completion(prompt : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "You will provide quizzes related to the topics middle school students have learned."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = QuizResponse,
    )
    return response.choices[0].message.parsed

In [75]:
response = completion(prompt)

In [66]:
response

QuizResponse(goal=['계속적 용법의 관계대명사를 사용하여 문장을 확장할 수 있어요.', '관련 부가 정보를 제공하는 관계대명사 who/which/that의 사용법을 익혀요.'], preview_quizzes=[PreviewQuiz(fill_in_blank_quiz='I met John, ___ gave me a great idea.', korean_translation='나는 존을 만났는데, 그가 나에게 훌륭한 아이디어를 주었어.', options=['which', 'who', 'whom'], answer='who', answer_index=2), PreviewQuiz(fill_in_blank_quiz='I met a famous writer, ___ gave me his autograph.', korean_translation='나는 유명한 작가를 만났는데, 그가 나에게 사인을 줬어.', options=['that', 'whom', 'who'], answer='who', answer_index=3)], sample_script=[SampleScript(sample_script_eng='A: I met John, who shared his experience with me. B: That sounds interesting. A: I met a brilliant artist, who inspired my creativity. B: Amazing!', sample_script_kor='A: 나는 존을 만났는데, 그가 자신의 경험을 나와 나눴어. B: 정말 흥미롭네. A: 나는 뛰어난 예술가를 만났는데, 그가 내 창의력에 영감을 줬어. B: 대단해!')])

In [29]:
# PreviewQuiz

In [76]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent = 4)
print(response_output)

{
    "goal": [
        "동격을 이용하여 of와 that의 용법을 이해할 수 있어요.",
        "동격의 표현, 즉 of와 that을 사용한 문장을 이해할 수 있어요."
    ],
    "preview_quizzes": [
        {
            "fill_in_blank_quiz": "The fact ___ his success surprised everyone.",
            "korean_translation": "그의 성공이라는 사실이 모두를 놀라게 했다.",
            "options": [
                "that",
                "of",
                "with"
            ],
            "answer": "of",
            "answer_index": 2
        },
        {
            "fill_in_blank_quiz": "The fact ___ he won surprised everyone.",
            "korean_translation": "그가 이겼다는 사실이 모두를 놀라게 했다.",
            "options": [
                "that",
                "if",
                "with"
            ],
            "answer": "that",
            "answer_index": 1
        }
    ],
    "sample_script": [
        {
            "sample_script_eng": "A: The fact of their achievement amazed me. B: Really? And the fact that she won also impressed everyone. A: I also discussed

In [31]:
print(response)

goal=['간단한 인사말을 활용할 수 있어요.', '만나거나 헤어질 때 적절한 인사말을 사용할 수 있어요.'] preview_quizzes=[PreviewQuiz(fill_in_blank_quiz='___ there!', korean_translation='안녕하세요!', options=['Hello', 'Hey', 'Hi'], answer='Hello', answer_index=1), PreviewQuiz(fill_in_blank_quiz='Hi! ___!', korean_translation='안녕! 잘 가요!', options=['See you', 'Goodbye', 'Farewell'], answer='Goodbye', answer_index=2)] sample_script=[SampleScript(sample_script_eng='A: Hey there! B: Hi! A: Goodbye! B: Take care!', sample_script_kor='A: 안녕! B: 안녕! A: 잘 가요! B: 조심해요!')]


# for문 돌려서 Batch API jsonl 파일 형식 만들기

확인 사항
- A1, A2, B1, B2, C1, C2 CEFR level 별로 퀴즈만 만들어지면 되는건가?
- 여섯개의 레벨 모두 만들어야하는지, 일부 레벨(A1, B1, C1)만 만들면 되는지도 결정

1. 커리큘럼 정보 csv 파일 읽어오기
2. for문으로 한줄씩 순회하면서 프롬프트 만들어주기들어주기
3. 렌더링한 프롬프트 파일에 써주기 (처음엔 조금만, ex. 5개 커리큘럼)
4. openai batch API 요청해서 task_id 받아오기 
5. batch task 완료되면 결과 받아오고 검수하기

In [32]:
def QuizMake(data, output_filename):
    jsonl_data = []

    for i in range(len(data)):
        prompt = prompt_template.render(
            description = data.loc[i,"DESCRIPTION"],
            key_expression = data.loc[i,"KEY_EXPRESSION"],
            sub_expression = [data.loc[i,"SUB_EXPRESSION"]],
        )

        quiz_request = {
            "custom_id" : f"request-{i+1}",
            "method" : "POST",
            "url" : "/v1/chat/completions",
            "body" : {
                "model" : "o3-mini-2025-01-31",
                "messages" : [
                    {"role": "system", "content": "You will provide quizzes related to the topics students have learned, appropriately matching their CEFR level for middle school students."},
                    {"role": "user", "content": prompt}
                ],
                "response_format" : response_format
            }
        }

        jsonl_data.append(quiz_request)

        with open(output_filename, 'w', encoding='utf-8') as jsonl_file:
            for item in jsonl_data:
                jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f'JSONL 파일 생성 완료 : {output_filename}-{i+1}')

## 초등학교 jsonl 파일

In [33]:
import pandas as pd

raw_data = pd.read_csv("../../../DB/Extract_Elem.csv")
data_Elem3 = raw_data.loc[raw_data['학년'] == '초3']
data_Elem4 = raw_data.loc[raw_data['학년'] == '초4']
data_Elem5 = raw_data.loc[raw_data['학년'] == '초5']
data_Elem6 = raw_data.loc[raw_data['학년'] == '초6']

# index 초기화화
data_Elem3 = data_Elem3.reset_index(drop=True)
data_Elem4 = data_Elem4.reset_index(drop=True)
data_Elem5 = data_Elem5.reset_index(drop=True)
data_Elem6 = data_Elem6.reset_index(drop=True)

display(data_Elem3.head(3))
display(data_Elem4.head(3))
display(data_Elem5.head(3))
display(data_Elem6.head(3))

,Unnamed: 0,textbook ID,학년,출판사,교육과정,단원 번호,단원 명,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,0,310,초3,YBM(김),2022,1,Hello! I’m Dito,Hello/Hi! 등의 표현을 사용해 만날 때 인사하기.,Hi there!,NaN,Hello! Hey!
1,1,310,초3,YBM(김),2022,2,"Sit Down, Please",극존칭 표현을 통해 요청하고 그에 긍정적으로 응답하기.,"A: Sit down, please. B: Sure.",NaN,A: Please take a seat. B: Certainly. A: Kindly...
2,2,310,초3,YBM(김),2022,3,What’s This?,의문문을 활용해 무슨 물건인지 묻고 답하기.,A: What's this? B: It's a box.,NaN,A: What's that? B: It's a (). A: What is this ...


,Unnamed: 0,textbook ID,학년,출판사,교육과정,단원 번호,단원 명,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,46,410,초4,YBM(김),2022,1,How Are You?,상대방의 안부를 물어보고 응답하기.,A: How are you doing? B: I'm doing fine.,NaN,A: How have you been? B: I'm feeling (). A: Is...
1,47,410,초4,YBM(김),2022,2,Let’s Play Baseball,Let's를 활용해 제안하고 이에 대해 답하기.,Let's go to the park.,NaN,Let's () together. How about we ()?
2,48,410,초4,YBM(김),2022,3,Where Is My Bag?,위치 묻기 표현을 활용하여 물건의 위치 묻고 답하기.,A: Where is my backpack? B: It's on the table.,NaN,A: Where is my pencil case? B: It's in the dra...


,Unnamed: 0,textbook ID,학년,출판사,교육과정,단원 번호,단원 명,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,92,503,초5,YBM(김),2015,1,How’s It Going?,How’s it going?을 활용해 안부 묻고 답하기.,How are you doing? — I'm doing well.,NaN,How have you been? — I've been doing well. How...
1,93,503,초5,YBM(김),2015,2,I’m in the Kitchen,의문사 Where를 활용해 사람의 위치 묻고 답하기.,A: Where are you? B: I'm in the living room.,NaN,A: Where is he? B: He's in the () room. A: Whe...
2,94,503,초5,YBM(김),2015,3,Whose Balloon Is This?,Whose의 소유격 의문사를 활용해 물건의 주인이 누구인지 묻고 답하기.,A: Whose bag is this? B: It's Donna's.,NaN,A: To whom does this belong? B: It's ()'s. A: ...


,Unnamed: 0,textbook ID,학년,출판사,교육과정,단원 번호,단원 명,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,119,603,초6,YBM(김),2015,1,What Grade Are You In?,What grade are you in? — I’m in the sixth grad...,A: What grade are you in? B: I'm in third grade.,NaN,A: What year are you in? B: I am in () grade. ...
1,120,603,초6,YBM(김),2015,2,What Would You Like?,'Would you like ()?'을 활용해 음식에 대해 묻고 답하기.,A: What would you like? B: I'd like strawberry...,NaN,A: What do you want to eat? B: I'd like (). A:...
2,121,603,초6,YBM(김),2015,3,My Favorite Subject Is Science,선호하는 과목에 대해 묻고 답하기.,A: What’s your favorite school subject? B: My ...,NaN,A: What class do you enjoy the most? B: My fav...


In [34]:
QuizMake(data_Elem3, output_filename = 'PreviewQuiz_Elem3_batch.jsonl')
QuizMake(data_Elem4, output_filename = 'PreviewQuiz_Elem4_batch.jsonl')
QuizMake(data_Elem5, output_filename = 'PreviewQuiz_Elem5_batch.jsonl')
QuizMake(data_Elem6, output_filename = 'PreviewQuiz_Elem6_batch.jsonl')

JSONL 파일 생성 완료 : PreviewQuiz_Elem3_batch.jsonl-240
JSONL 파일 생성 완료 : PreviewQuiz_Elem4_batch.jsonl-231
JSONL 파일 생성 완료 : PreviewQuiz_Elem5_batch.jsonl-99
JSONL 파일 생성 완료 : PreviewQuiz_Elem6_batch.jsonl-100
